# COVID-19 air pollution project. 
## Finding D value
### Written by Bujin Bekbulat

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import scipy.stats as st
import scipy.io as sio
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import statsmodels.stats.multicomp as multi
import matplotlib.dates as md
import datetime
from math import sqrt
import plotly.graph_objects as go
%matplotlib inline
from scipy.stats import iqr
from statistics import median 
from math import sin, cos, sqrt, atan2, radians
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)


In [30]:
#Reading file with all the states name
states = pd.read_csv("/Users/Bujin/Downloads/States.csv")

#Reading dates
#dateparse = lambda dates: [pd.datetime.strptime(d, '%m/%d/%y') for d in dates]

#Defining output

#Data = pd.read_csv("/Users/Bujin/Downloads/Region_Mobility_Report_CSVs/2020_US_Region_Mobility_Report.csv",parse_dates=['Date'], date_parser=dateparse)
Data = pd.read_csv("/Users/Bujin/Downloads/Region_Mobility_Report_CSVs/2020_US_Region_Mobility_Report.csv")

Data['Date']=pd.to_datetime(Data['Date'])
Reopening= pd.read_excel("/Users/Bujin/reopening1.xlsx", date_parser=dateparse)
Reopening['Start'] = Reopening['Shut Down'].dt.dayofyear
Reopening['End'] = Reopening['Reopening '].dt.dayofyear

In [32]:

#Defining output
DF=pd.DataFrame()
Slope=pd.DataFrame()
OT=pd.DataFrame()
adjusted=[]
idw=[]
total_mon=0
DF5=pd.DataFrame()
Slope5=pd.DataFrame()
OT5=pd.DataFrame()
adjusted5=[]
idw5=[]

lastday=2
twoweeks=16
totallength=35
length=24
#Calling each state file

#Data = pd.read_csv("/Users/Bujin/Downloads/Historical PM2.5.csv",parse_dates=['Date'], date_parser=dateparse)
    
a=[x for x in  list(set(Data['census_fips_code'])) if pd.notnull(x)] #each unique monitors in each state
total_mon=total_mon+len(a) #Counting total monitors

    #Defining output
df=pd.DataFrame()
slope1=pd.DataFrame()
ot=pd.DataFrame()
df5=pd.DataFrame()
slope15=pd.DataFrame()
ot5=pd.DataFrame()
threshhold=0.75
    
    #for each station calculate D value   
for iii in range(0,len(a)):
       monitor=Data[Data['census_fips_code']==a[iii]]
             
       #Defining as variables
    
       state=list(monitor['sub_region_1'])[0] 
       county=list(monitor['sub_region_2'])[0]

       weeknum=0
       weeknum1=0
       weeknum2=0
       weeknum3=0
       #Defining output
       List10=[] 
       slope=[]  
       List5=[] 
       slope5=[]  
       start1=Reopening[Reopening['State']==str(state)]['Shut Down']
       print(start1)
       start1=start1.reset_index(drop=True)
       start1=start1[0]
       end1=Reopening[Reopening['State']==str(state)]['Reopening ']
       end1=end1.reset_index(drop=True)
       end1=end1[0]
       sta=monitor[monitor['Date']>=start1] ##Change this date (add 7 days each week)
       en=monitor[monitor['Date']<end1]
       sten=sta[sta['Date']<end1]
   
       #Starting filters
       
       #Days pre and post covid

       whole=monitor[monitor['Date']<pd.to_datetime('2020-09-'+str(lastday))] ##Change this date (add 7 days each week)
       twentytwenty=pd.concat([whole['Date'],whole['census_fips_code']],axis=1,sort=False)
       twentytwenty=twentytwenty.dropna()
       pre=twentytwenty[twentytwenty['Date']<pd.to_datetime('2020-03-19')]
       post=twentytwenty[twentytwenty['Date']>=pd.to_datetime('2020-03-19')]

       #Number of days for adjusting
       adjusting=monitor[monitor['Date']<pd.to_datetime('2020-09-'+str(twoweeks))] ## Change this date (add 7 days each week)
       
       #start1='2020-01-01'
       
       #Using monitors that have enough data
       if len(adjusting)>=totallength:  ##Change this (add 5 days on post and adjusting each week)
                                        
        for j in range(0,len(sten),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(start1) + datetime.timedelta(j)
          end_d = start_d + datetime.timedelta(7)
          week=monitor[monitor['Date']>=start_d]
          if end_d>end1:
            end_d=end1
          week=week[week['Date']<end_d]
          #print(week)
          if len(week)>0: 
           #Week 
           twenty=pd.DataFrame(week["transit_stations_percent_change_from_baseline"])
           weeknum=weeknum+1 #Week #
                
                  #Appending for output
           List10.append([np.nanmedian(twenty),("Week "+str(weeknum)),state,county,a[iii]])
                  
        for j in range(0,len(sta),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(start1) - datetime.timedelta(j)-datetime.timedelta(7)
          end_d = start_d + datetime.timedelta(7)
          #print('BEFORE')
          #print(start_d)
                    
          week=monitor[monitor['Date']>=start_d]
          week=week[week['Date']<end_d]
          #print(week)
          if len(week)>0: 
           #Week 
           twenty=pd.DataFrame(week["transit_stations_percent_change_from_baseline"])
                
                  #Appending for output
           List10.append([np.nanmedian(twenty),("Week "+str(weeknum1)),state,county,a[iii]])
                 
        for j in range(0,len(whole)-len(en),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(end1) + datetime.timedelta(j)
          end_d = start_d + datetime.timedelta(7)
                   
          week=monitor[monitor['Date']>=start_d]
          week=week[week['Date']<end_d]
          #print(week)
          if len(week)>0: 
           #Week 
           twenty=pd.DataFrame(week["transit_stations_percent_change_from_baseline"])
           weeknum2=weeknum2+1 #Week #
                
                  #Appending for output
           List10.append([np.nanmedian(twenty),("Week +"+str(weeknum2)),state,county,a[iii]])
                  

        for j in range(0,len(sten),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(end1) - datetime.timedelta(j)-datetime.timedelta(7)
          end_d = start_d + datetime.timedelta(7)         
          week=monitor[monitor['Date']>=start_d]
          week=week[week['Date']<end_d]
          if start_d<start1:
                start_d=start1
          #print(week)
          if len(week)>0: 
           #Week 
           twenty=pd.DataFrame(week["transit_stations_percent_change_from_baseline"])
           weeknum3=weeknum3-1 #Week #
                  #Appending for output
           List10.append([np.nanmedian(twenty),("Week -"+str(weeknum3)),state,county,a[iii]])
        List10=pd.DataFrame(List10)
        df=pd.concat([df, List10], axis=0, sort=False)      
  
    #Concating for output
DF=pd.concat([DF, df], axis=0, sort=False)


34   2020-03-23
Name: Shut Down, dtype: datetime64[ns]


/Users/Bujin/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning:

All-NaN slice encountered



21   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
28   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
34   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
34   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
34   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
49   NaT
Name: Shut Down, dtype: datetime64[ns]
49   NaT
Name: Shut Down, dtype: datetime64[ns]
49   NaT
Name: Shut Down, dtype: datetime64[ns]
42   2020-04-02
Name: Shut Down, dtype: datetime64[ns]
0   2020-04-04
Name: Shut Down,

31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
15   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-03-22
Name: Shut Down, dtype: datetime64[ns]
31   2020-

46   2020-03-25
Name: Shut Down, dtype: datetime64[ns]
46   2020-03-25
Name: Shut Down, dtype: datetime64[ns]
46   2020-03-25
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
35   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: datetime64[ns]
3   NaT
Name: Shut Down, dtype: dat

...

17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-03-23
Name: Shut Down, dtype: datetime64[ns]
17   2020-

19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
46   2020-03-25
Name: Shut Down, dtype: datetime64[ns]
19   2020-03-30
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
Name: Shut Down, dtype: datetime64[ns]
20   2020-03-24
N

22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-03-27
Name: Shut Down, dtype: datetime64[ns]
22   2020-

24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-04-06
Name: Shut Down, dtype: datetime64[ns]
24   2020-

43   NaT
Name: Shut Down, dtype: datetime64[ns]
39   2020-04-07
Name: Shut Down, dtype: datetime64[ns]
39   2020-04-07
Name: Shut Down, dtype: datetime64[ns]
42   2020-04-02
Name: Shut Down, dtype: datetime64[ns]
14   NaT
Name: Shut Down, dtype: datetime64[ns]
14   NaT
Name: Shut Down, dtype: datetime64[ns]
14   NaT
Name: Shut Down, dtype: datetime64[ns]
14   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut Down, dtype: datetime64[ns]
26   NaT
Name: Shut

In [37]:
DF[DF[4]==1029]

,0,1,2,3,4
0,-34.0,Week 1,Alabama,Cleburne County,1029.0
1,-22.0,Week 2,Alabama,Cleburne County,1029.0
2,-18.0,Week 3,Alabama,Cleburne County,1029.0
3,-15.0,Week 4,Alabama,Cleburne County,1029.0
4,-23.0,Week 0,Alabama,Cleburne County,1029.0
5,-19.0,Week 0,Alabama,Cleburne County,1029.0
6,-9.0,Week 0,Alabama,Cleburne County,1029.0
7,9.0,Week 0,Alabama,Cleburne County,1029.0
8,5.0,Week 0,Alabama,Cleburne County,1029.0
9,-4.0,Week 0,Alabama,Cleburne County,1029.0


In [42]:
#Starting IDW
k1=pd.DataFrame(idw)
k1=k1.drop_duplicates()
k1.columns=['ID','Latitude','Longitude']
k1=k1.reset_index(drop=True)
IDW= list(k1['ID'])

k2=pd.DataFrame(adjusted)
k2.columns=['ID','Latitude','Longitude','Slope','Week']
k2=k2.drop(['Slope'],axis=1)
k2=k2.drop(['Week'],axis=1)
k2=k2.drop_duplicates()
adjusted=pd.DataFrame(adjusted)
adjusted.columns=['ID','Latitude','Longitude','Slope','Week']
k2.dropna()
k2=k2.drop_duplicates()

In [43]:
#IDW with nearest stations in 50km
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('haversine')
distance=pd.DataFrame(dist.pairwise(k2[['Latitude','Longitude']].to_numpy(), k1[['Latitude','Longitude']].to_numpy())*6373,columns=k1["ID"], index=k2["ID"])

In [19]:
start1

Series([], Name: Shut Down, dtype: datetime64[ns])

In [20]:
#IDW
Slope1=pd.DataFrame()
w=[]

for i in range (0, len(IDW)):
    
    monitor=IDW[i]
    s=distance[monitor]
    
    #find 3 nearest monitors in 50 km
    nearest=s[s<50]
    nearest=nearest[nearest>0]
    nearest=nearest.sort_values(ascending=True)
    nearest=nearest[:3]
    idw=pd.DataFrame()
    print(nearest)
    #If there are monitors
    if len(nearest)!= 0:
        n=0
        
        o=list(nearest)
        
        #For each of the nearest monitors finding slope and distance:
        for j in range (0,len(nearest)):
            
            nearest_mon=nearest.index[j]
            nearest_slope=list(Slope[str(nearest_mon)])
            #print(pd.DataFrame(nearest_slope))
            nearest_slope.append(1/nearest.iloc[j])
            nearest_slope=pd.DataFrame(nearest_slope)
 #           nearest_slope=pd.concat([nearest_slope,Slope['Week']],axis=1,sort=False)
            #print(nearest_slope)
            nearest_slope.columns=[n]
            
            idw=pd.concat([idw,nearest_slope],axis=1,sort=False)
            #print(idw)
            n=n+1
            
            length=len(list(idw[0]))
            idw_df=pd.DataFrame()
            df=pd.DataFrame()
            #print(pd.DataFrame(nearest_slope))
            #Dataframing slope for nearest monitor and inverse distance to that monitor
            for l in range (0,len(idw.columns)):
                 t=[]      
                 for m in range (0,length-1):

                         t.append([list(idw[l])[m]*list(idw[l])[length-1],list(idw[l])[length-1]])
                       #  print(list(idw[l])[m])
                 t=pd.DataFrame(t)

                 df=pd.concat([df,t],axis=1,sort=False) 
                 df=pd.DataFrame(df)
                 
 
        idw_df=pd.concat([idw_df,df],axis=1,sort=False)
        idw_df=idw_df.transpose()
        print(idw_df)
        slp=[]
        
        #Aggregating nearest monitor slopes and distance 
        for z in range(0,len(idw_df.columns)):
                row=list(idw_df[z])
                nom=0
                denom=0
                for y in range (0,len(row)-1,2):
                    if pd.notnull(row[y]):
                     nom=nom+row[y]
                     denom=denom+row[y+1]

                if denom!=0:
                 frac=nom/denom
                 slp.append(frac)
                else:
                 slp.append(np.nan)  

        slp=pd.DataFrame(slp)
        slp.columns=[monitor]

        #Output 
        Slope1=pd.concat([Slope1,slp],axis=1,sort=False)
Slope1=pd.concat([Slope1,Slope['Week']],axis=1,sort=False)
        

NameError: name 'IDW' is not defined

In [46]:
Slope.d=Slope1[Slope1['Week']>='Week 1']
Slope.d=Slope.d.reset_index(drop=True)
Slope.b=Slope1[Slope1['Week']>='Week -1']
Slope.b=Slope.b[Slope.b['Week']<'Week 1']
Slope.b=Slope.b.reset_index(drop=True)
Slope.a=Slope1[Slope1['Week']<='Week +9']
Slope.a=Slope.a.reset_index(drop=True)
Slope.bb=Slope1[Slope1['Week']<='Week --9']
Slope.bb=Slope.bb[Slope.bb['Week']>'Week +9']
Slope.bb=Slope.bb.reset_index(drop=True)
Slope.bb

/Users/Bujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/Users/Bujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/Users/Bujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/Users/Bujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



,391510016,250112005,391510022,361173001,420010001,390570006,390271002,260991003,421255200,330111011,...,180910010,290990019,450030003,240130001,560252601,80130014,390490081,390230001,390230003,Week
0,-0.000232,-0.000367,-0.000345,-0.000714,-0.000232,-0.000543,-0.000354,-0.000213,-0.000627,-0.000136,...,0.000712,0.000301,-0.000460,-0.000335,0.000083,-0.000469,-0.000741,-0.000576,-0.000574,Week --1
1,-0.000169,-0.000447,-0.000311,-0.000864,-0.000399,-0.000209,-0.000196,-0.000278,-0.000506,-0.000360,...,0.000346,-0.000121,-0.000277,-0.000420,-0.000119,-0.000407,-0.000429,-0.000158,-0.000228,Week --2
2,-0.000246,-0.000513,-0.000377,-0.000573,-0.000569,-0.000341,-0.000242,-0.000406,-0.000489,-0.000404,...,0.000329,0.000021,-0.000407,-0.000591,NaN,-0.000338,-0.000592,-0.000269,-0.000345,Week --3
3,-0.000281,-0.000431,-0.000361,-0.000486,-0.000362,-0.000524,-0.000354,-0.000507,-0.000552,-0.000453,...,0.000475,0.000097,NaN,-0.000424,-0.000256,-0.000555,-0.000663,-0.000510,-0.000557,Week --4
4,-0.000257,-0.000309,-0.000279,-0.000268,-0.000336,-0.000537,-0.000434,-0.000036,-0.000556,-0.000354,...,-0.000070,0.000455,NaN,-0.000315,-0.000179,-0.000533,-0.000720,-0.000512,-0.000566,Week --5
5,-0.000488,-0.000185,-0.000471,-0.000214,-0.000161,-0.000726,-0.000692,-0.000101,-0.000476,-0.000364,...,-0.000074,0.000464,NaN,-0.000487,0.000083,NaN,-0.000874,-0.000664,-0.000767,Week --6
6,-0.000927,-0.000197,-0.000948,-0.000068,-0.000262,-0.000883,-0.000762,-0.000283,-0.000659,-0.000232,...,NaN,0.000055,NaN,-0.000921,-0.000345,NaN,-0.001207,-0.000919,-0.000948,Week --7
7,-0.001486,-0.000085,-0.001296,-0.000232,NaN,-0.001659,-0.001593,-0.000187,NaN,0.000009,...,NaN,-0.000318,NaN,NaN,-0.000089,NaN,-0.001684,-0.001567,-0.001729,Week --8
8,NaN,NaN,NaN,-0.000127,NaN,NaN,NaN,-0.000563,NaN,NaN,...,NaN,0.000109,NaN,NaN,0.000214,NaN,NaN,NaN,NaN,Week --9
9,NaN,NaN,NaN,-0.000068,NaN,NaN,NaN,-0.000719,NaN,NaN,...,NaN,-0.000400,NaN,NaN,-0.000173,NaN,NaN,NaN,NaN,Week --10


In [47]:
#Finding D value from IDW

#Reading file with all the states name
states = pd.read_csv("/Users/Bujin/Downloads/States.csv")

#Reading dates
dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d') for d in dates]

#Defining output
DF1=pd.DataFrame()
Slope2=pd.DataFrame()
OT1=pd.DataFrame()
adjusted=[]
idw=[]


#Calling each state file

Data = pd.read_csv("/Users/Bujin/Downloads/Historical PM2.5.csv",parse_dates=['Date'], date_parser=dateparse)


a=set(Data['ID']).intersection(Slope1.columns) #Taking the intersection of IDW monitors and all monitors
a=list(a)

    #Defining output
df=pd.DataFrame()
slope1=pd.DataFrame()
ot=pd.DataFrame()
    
    #for each station calculate D value   
for iii in range(0,len(a)):
       monitor=Data[Data['ID']==a[iii]]
       
       #Defining as variables
       site=list(monitor['Site'])[1]        
       state=list(monitor['State'])[1] 
       idd=list(monitor['ID'])[1]
       county=list(monitor['County'])[1]
       lon=(list(monitor['Longitude'])[1])
       lat=(list(monitor['Latitude'])[1])
       weeknum=0
       weeknum1=0
       weeknum2=0
       weeknum3=0
       start1=Reopening[Reopening['State']==str(state)]['Shut Down']
       start1=start1.reset_index(drop=True)
       start1=start1[0]
       end1=Reopening[Reopening['State']==str(state)]['Reopening ']
       end1=end1.reset_index(drop=True)
       end1=end1[0]
       sta=monitor[monitor['Date']>=start1] ##Change this date (add 7 days each week)
       en=monitor[monitor['Date']<end1]
       sten=sta[sta['Date']<end1]


 
       #Defining output
       List10=[] 
       slope=[]  
    
       #Starting filters
       

       whole=monitor[monitor['Date']<pd.to_datetime('2020-09-'+str(lastday))] ##Change this date (add 7 days each week)
       twentytwenty=pd.concat([whole['Date'],whole['2020']],axis=1,sort=False)
       twentytwenty=twentytwenty.dropna()
       pre=twentytwenty[twentytwenty['Date']<pd.to_datetime('2020-03-19')]
       post=twentytwenty[twentytwenty['Date']>=pd.to_datetime('2020-03-19')]


       #Number of days for adjusting
       adjusting=monitor[monitor['Date']<pd.to_datetime('2020-09-'+str(twoweeks))] ## Change this date (add 7 days each week)

       #dropping years if it has incomplete data
       drop=adjusting.drop(['2020'],axis=1)
       drop=drop.dropna(thresh=0.75*len(adjusting), axis=1)
       
       #checking if it has 2 years out of last 3 years
       Iqr=pd.concat([adjusting['2019'],adjusting['2018'],adjusting['2017']],axis=1,sort=False)
       Iqr = Iqr.dropna(thresh=0.75*len(Iqr), axis=1)
       
       
       
       #start='2020-01-01'
       
       #Using monitors that have enough data
       if len(drop.columns)>=10 and len(post)>=length  and len(adjusting)>=totallength and len(pre)>=59 and len(Iqr.columns)>=2:   ##Change this (add 5 days on post and adjusting each week)
                                        
                                        
        for j in range(0,len(sten),7): # Week by week
            
           #Start and end date for each week
           start_d = pd.to_datetime(start1) + datetime.timedelta(j)
           end_d = start_d + datetime.timedelta(7)

         
           week=monitor[monitor['Date']>=start_d]
           week=week[week['Date']<end_d]
            
           #Week 
           twenty=pd.DataFrame(week["2020"])
           weeknum=weeknum+1 #Week #
           
           #Month (for adjusting)
           end_date = list(week['Date'])[0] + datetime.timedelta(days=21) # +2 weeks
           start_date = list(week['Date'])[0] - datetime.timedelta(days=14) # -2 weeks

        
           # First 2 weeks of the year, add december data from previous year
           if pd.to_datetime('2020-01-01')== list(week['Date'])[0] :
               half=monitor[monitor["Date"]>='2020-12-18']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-22']
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011","2009":'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           elif list(week['Date'])[0]==pd.to_datetime('2020-01-08'):
               half=monitor[monitor["Date"]>'2020-12-24']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-29'] 
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011",'2009':'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           else:
               month=monitor[monitor["Date"]>=start_date]
               month=month[month["Date"]<end_date]
               month=month.drop(['2009'],axis=1)
        
           #if len(month.columns)==19:
               #month=month.drop(['2009'],axis=1)
           
           Mnth=month.drop(['2020'], axis=1)
           
           #Change the 2020 data to only that particular week
           month["2020"]=twenty
           
           #Dropping columns to get only historical data
           months=month.drop(['2020'],axis=1)
           months=months.drop(['Site'],axis=1)
           months=months.drop(['State'],axis=1)
           months=months.drop(['Longitude'],axis=1)          
           months=months.drop(['Latitude'],axis=1)
           months=months.drop(['Date'],axis=1)
           months=months.drop(['ID'],axis=1)
           months=months.drop(['County'],axis=1)
           
           #Filtering data, using years that have enough data
           historical=months.dropna(thresh=0.75*len(months),axis=1)
          # historical=months.dropna(axis=1,how='all')
           #Finding median for each year
           medians=np.nanmedian(historical,axis=0)
           
           #Finding iqr
           IQR_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=['2019', '2018', '2017'])
           IQR_melt=IQR_melt.dropna()
           IQR_melt.columns = ['index', 'year', 'value']
           
           #Finding slope of historical data
           hist_slope=pd.DataFrame({"x":pd.to_numeric(historical.columns),"y":np.array(medians)})
           x=hist_slope["x"]
           y=hist_slope["y"]
        
           #Finding historical median
           
           Historical_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=historical.columns)
           Historical_melt.columns = ['index', 'year', 'value']
           Iqr=pd.concat([adjusting['2019'],adjusting['2018'],adjusting['2017']],axis=1,sort=False)
           Iqr = Iqr.dropna(thresh=0.75*len(Iqr), axis=1)


                  
                  #10 year median 
           Med=np.nanmedian(Historical_melt['value'])  
                  
                  #Adjusting historical median with IDW slope
           adjusted_med =  list(Slope.d[idd])[weeknum-1] * (len(historical.columns)+1)/2 + Med

                  #Calculating D value 
           D_value=(np.nanmedian(month['2020'])-adjusted_med)/iqr(IQR_melt['value'])  
                
                  #Appending for output
           List10.append([D_value,np.nanmedian(month['2020']),Med,adjusted_med,list(Slope.d[idd])[weeknum-1],999,("Week "+str(weeknum)),state,idd,county,lat,lon,999])
           slope.append(list(Slope.d[idd])[weeknum-1])       
        for j in range(0,len(sta),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(start1) - datetime.timedelta(j)-datetime.timedelta(7)
          end_d = start_d + datetime.timedelta(7)

         
          week=monitor[monitor['Date']>=start_d]
          week=week[week['Date']<end_d]
            
           #Week 
          if len(week)>0:
           twenty=pd.DataFrame(week["2020"])
           weeknum1=weeknum1-1 #Week #
           weeknum=weeknum+1
           #Month (for adjusting)
           end_date = list(week['Date'])[0] + datetime.timedelta(days=21) # +2 weeks
           start_date = list(week['Date'])[0] - datetime.timedelta(days=14) # -2 weeks

        
           # First 2 weeks of the year, add december data from previous year
           if pd.to_datetime('2020-01-01')== list(week['Date'])[0] :
               half=monitor[monitor["Date"]>='2020-12-18']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-22']
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011","2009":'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           elif list(week['Date'])[0]==pd.to_datetime('2020-01-08'):
               half=monitor[monitor["Date"]>'2020-12-24']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-29'] 
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011",'2009':'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           else:
               month=monitor[monitor["Date"]>=start_date]
               month=month[month["Date"]<end_date]
               month=month.drop(['2009'],axis=1)
        
           #if len(month.columns)==19:
               #month=month.drop(['2009'],axis=1)
           
           Mnth=month.drop(['2020'], axis=1)
           
           #Change the 2020 data to only that particular week
           month["2020"]=twenty
           
           #Dropping columns to get only historical data
           months=month.drop(['2020'],axis=1)
           months=months.drop(['Site'],axis=1)
           months=months.drop(['State'],axis=1)
           months=months.drop(['Longitude'],axis=1)          
           months=months.drop(['Latitude'],axis=1)
           months=months.drop(['Date'],axis=1)
           months=months.drop(['ID'],axis=1)
           months=months.drop(['County'],axis=1)
           
           #Filtering data, using years that have enough data
           historical=months.dropna(thresh=0.75*len(months),axis=1)
          # historical=months.dropna(axis=1,how='all')
           #Finding median for each year
           medians=np.nanmedian(historical,axis=0)
           
           #Finding iqr
           IQR_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=['2019', '2018', '2017'])
           IQR_melt=IQR_melt.dropna()
           IQR_melt.columns = ['index', 'year', 'value']
           
           #Finding slope of historical data
           hist_slope=pd.DataFrame({"x":pd.to_numeric(historical.columns),"y":np.array(medians)})
           x=hist_slope["x"]
           y=hist_slope["y"]
        
           #Finding historical median
           
           Historical_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=historical.columns)
           Historical_melt.columns = ['index', 'year', 'value']
           Iqr=pd.concat([adjusting['2019'],adjusting['2018'],adjusting['2017']],axis=1,sort=False)
           Iqr = Iqr.dropna(thresh=0.75*len(Iqr), axis=1)


                  
                  #10 year median 
           Med=np.nanmedian(Historical_melt['value'])  
                  
                  #Adjusting historical median with IDW slope
           adjusted_med =  list(Slope.b[idd])[(-1)*weeknum1-1] * (len(historical.columns)+1)/2 + Med

                  #Calculating D value 
           D_value=(np.nanmedian(month['2020'])-adjusted_med)/iqr(IQR_melt['value'])  
                
                  #Appending for output
           List10.append([D_value,np.nanmedian(month['2020']),Med,adjusted_med,list(Slope.b[idd])[-1*weeknum1-1],999,("Week "+str(weeknum1)),state,idd,county,lat,lon,999])
           slope.append(list(Slope1[idd])[weeknum-1])       
        for j in range(0,len(whole)-len(en),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(end1) + datetime.timedelta(j)
          end_d = start_d + datetime.timedelta(7)

         
          week=monitor[monitor['Date']>=start_d]
          week=week[week['Date']<end_d]
            
           #Week 
          if len(week)>0:
           twenty=pd.DataFrame(week["2020"])
           weeknum2=weeknum2+1 #Week #
           weeknum=weeknum+1
           #Month (for adjusting)
           end_date = list(week['Date'])[0] + datetime.timedelta(days=21) # +2 weeks
           start_date = list(week['Date'])[0] - datetime.timedelta(days=14) # -2 weeks

        
           # First 2 weeks of the year, add december data from previous year
           if pd.to_datetime('2020-01-01')== list(week['Date'])[0] :
               half=monitor[monitor["Date"]>='2020-12-18']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-22']
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011","2009":'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           elif list(week['Date'])[0]==pd.to_datetime('2020-01-08'):
               half=monitor[monitor["Date"]>'2020-12-24']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-29'] 
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011",'2009':'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           else:
               month=monitor[monitor["Date"]>=start_date]
               month=month[month["Date"]<end_date]
               month=month.drop(['2009'],axis=1)
        
           #if len(month.columns)==19:
               #month=month.drop(['2009'],axis=1)
           
           Mnth=month.drop(['2020'], axis=1)
           
           #Change the 2020 data to only that particular week
           month["2020"]=twenty
           
           #Dropping columns to get only historical data
           months=month.drop(['2020'],axis=1)
           months=months.drop(['Site'],axis=1)
           months=months.drop(['State'],axis=1)
           months=months.drop(['Longitude'],axis=1)          
           months=months.drop(['Latitude'],axis=1)
           months=months.drop(['Date'],axis=1)
           months=months.drop(['ID'],axis=1)
           months=months.drop(['County'],axis=1)
           
           #Filtering data, using years that have enough data
           historical=months.dropna(thresh=0.75*len(months),axis=1)
          # historical=months.dropna(axis=1,how='all')
           #Finding median for each year
           medians=np.nanmedian(historical,axis=0)
           
           #Finding iqr
           IQR_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=['2019', '2018', '2017'])
           IQR_melt=IQR_melt.dropna()
           IQR_melt.columns = ['index', 'year', 'value']
           
           #Finding slope of historical data
           hist_slope=pd.DataFrame({"x":pd.to_numeric(historical.columns),"y":np.array(medians)})
           x=hist_slope["x"]
           y=hist_slope["y"]
        
           #Finding historical median
           
           Historical_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=historical.columns)
           Historical_melt.columns = ['index', 'year', 'value']
           Iqr=pd.concat([adjusting['2019'],adjusting['2018'],adjusting['2017']],axis=1,sort=False)
           Iqr = Iqr.dropna(thresh=0.75*len(Iqr), axis=1)


                  
                  #10 year median 
           Med=np.nanmedian(Historical_melt['value'])  
                  
                  #Adjusting historical median with IDW slope
           adjusted_med =  list(Slope.a[idd])[weeknum2-1] * (len(historical.columns)+1)/2 + Med

                  #Calculating D value 
           D_value=(np.nanmedian(month['2020'])-adjusted_med)/iqr(IQR_melt['value'])  
                
                  #Appending for output
           List10.append([D_value,np.nanmedian(month['2020']),Med,adjusted_med,list(Slope.a[idd])[weeknum2-1],999,("Week +"+str(weeknum2)),state,idd,county,lat,lon,999])
           slope.append(list(Slope.a[idd])[weeknum2-1])       
        for j in range(0,len(sten),7): # Week by week
            
           #Start and end date for each week
          start_d = pd.to_datetime(start1) - datetime.timedelta(j)-datetime.timedelta(7)
          end_d = start_d + datetime.timedelta(7)

         
          week=monitor[monitor['Date']>=start_d]
          week=week[week['Date']<end_d]
            
           #Week 
          if len(week)>0:
           twenty=pd.DataFrame(week["2020"])
           weeknum3=weeknum3-1 #Week #
           weeknum=weeknum+1
           #Month (for adjusting)
           end_date = list(week['Date'])[0] + datetime.timedelta(days=21) # +2 weeks
           start_date = list(week['Date'])[0] - datetime.timedelta(days=14) # -2 weeks

        
           # First 2 weeks of the year, add december data from previous year
           if pd.to_datetime('2020-01-01')== list(week['Date'])[0] :
               half=monitor[monitor["Date"]>='2020-12-18']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-22']
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011","2009":'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           elif list(week['Date'])[0]==pd.to_datetime('2020-01-08'):
               half=monitor[monitor["Date"]>'2020-12-24']
               month=monitor[monitor["Date"]>='2020-01-01']
               month=month[month['Date']<'2020-01-29'] 
               half.rename(columns={"2020": "2009", "2019": "2020","2018":"2019","2017":"2018","2016":"2017","2015":"2016","2014":"2015","2013":"2014","2012":"2013","2011":"2012","2010":"2011",'2009':'2010'}, inplace=True)
               month = pd.concat((month, half), axis=0)
               month=month.drop(['2009'],axis=1)
           else:
               month=monitor[monitor["Date"]>=start_date]
               month=month[month["Date"]<end_date]
               month=month.drop(['2009'],axis=1)
        
           #if len(month.columns)==19:
               #month=month.drop(['2009'],axis=1)
           
           Mnth=month.drop(['2020'], axis=1)
           
           #Change the 2020 data to only that particular week
           month["2020"]=twenty
           
           #Dropping columns to get only historical data
           months=month.drop(['2020'],axis=1)
           months=months.drop(['Site'],axis=1)
           months=months.drop(['State'],axis=1)
           months=months.drop(['Longitude'],axis=1)          
           months=months.drop(['Latitude'],axis=1)
           months=months.drop(['Date'],axis=1)
           months=months.drop(['ID'],axis=1)
           months=months.drop(['County'],axis=1)
           
           #Filtering data, using years that have enough data
           historical=months.dropna(thresh=0.75*len(months),axis=1)
          # historical=months.dropna(axis=1,how='all')
           #Finding median for each year
           medians=np.nanmedian(historical,axis=0)
           
           #Finding iqr
           IQR_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=['2019', '2018', '2017'])
           IQR_melt=IQR_melt.dropna()
           IQR_melt.columns = ['index', 'year', 'value']
           
           #Finding slope of historical data
           hist_slope=pd.DataFrame({"x":pd.to_numeric(historical.columns),"y":np.array(medians)})
           x=hist_slope["x"]
           y=hist_slope["y"]
        
           #Finding historical median
           
           Historical_melt = pd.melt(month.reset_index(), id_vars=['index'], value_vars=historical.columns)
           Historical_melt.columns = ['index', 'year', 'value']
           Iqr=pd.concat([adjusting['2019'],adjusting['2018'],adjusting['2017']],axis=1,sort=False)
           Iqr = Iqr.dropna(thresh=0.75*len(Iqr), axis=1)


                  
                  #10 year median 
           Med=np.nanmedian(Historical_melt['value'])  
                  
                  #Adjusting historical median with IDW slope
           adjusted_med =  list(Slope.bb[idd])[(-1)*weeknum3-1] * (len(historical.columns)+1)/2 + Med

                  #Calculating D value 
           D_value=(np.nanmedian(month['2020'])-adjusted_med)/iqr(IQR_melt['value'])  
                
                  #Appending for output
           List10.append([D_value,np.nanmedian(month['2020']),Med,adjusted_med,list(Slope.bb[idd])[-1*weeknum3-1],999,("Week -"+str(weeknum3)),state,idd,county,lat,lon,999])



                
        slope=pd.DataFrame(slope)
        
        #Preparing for idw
        
        if slope.size>0:
           slope.columns=[str(idd)]
         

        ot=pd.concat([ot,monitor],axis=0,sort=False)
        slope1=pd.concat([slope1,slope],axis=1,sort=False)   
        List10=pd.DataFrame(List10)
        df=pd.concat([df, List10], axis=0, sort=False)      

    #Concating for output
       
OT1=pd.concat([OT1,ot],axis=0,sort=False)
DF1=pd.concat([DF1, df], axis=0, sort=False)
Slope2=pd.concat([Slope2,slope1],axis=1,sort=False)
    







/Users/Bujin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/Users/Bujin/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Bujin/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning:

All-NaN slice encountered

/Users/Bujin/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning:

Mean of empty slice



In [48]:
#Concat both outputs
MapData=pd.concat([DF,DF1],axis=0,sort=False)
MapData.columns=['Ratio','Median','Unadjusted','Expected','Slope','Rsquared','Week','state','ID','County','Latitude','Longitude','Pvalue']

In [49]:
Map=MapData.dropna()
ag = Map.groupby('ID')['Week'].nunique()
Dta=pd.DataFrame()

for i in range (0,len(ag.index)):
    a=ag.index[i]
    indexNames = MapData[ MapData['ID'] == a ].index
    #using station that have at least 75% of the weeks
    if ag.iloc[i]>28:
      Dta=pd.concat([Dta,MapData[MapData['ID']==a]])
Dta=Dta.dropna()
Dta.drop_duplicates(subset=['Week', 'ID'], inplace=True, keep='first')
Dta.to_csv(r"/Users/Bujin/Downloads/Ozone data non all.csv", index = False)


In [50]:
we=Dta['Week'].nunique()
Dta['ID'].nunique()

1039